<a href="https://colab.research.google.com/github/massenergize/rad/blob/hasha_refactor/merge-data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Understand the python environment

The google colab environment has a bunch of packages installed.  It's not using conda environments.

In [ ]:
from platform import python_version

print(python_version())

3.6.9


Python 3.6 will be end-of-life at the end of 2021, and lacks some important new features available in Python 3.7 and 3.8 for writing high-quality code (e.g. explicit type hinting).  It's fine to work with for now, but as we harden this we will want to migrate out of colab notebooks and update the code to v3.8 syntax

In [ ]:
! conda list

/bin/bash: conda: command not found


In [ ]:
! pip list

Package                       Version        
----------------------------- ---------------
absl-py                       0.10.0         
alabaster                     0.7.12         
albumentations                0.1.12         
altair                        4.1.0          
argon2-cffi                   20.1.0         
asgiref                       3.3.1          
astor                         0.8.1          
astropy                       4.1            
astunparse                    1.6.3          
async-generator               1.10           
atari-py                      0.2.6          
atomicwrites                  1.4.0          
attrs                         20.3.0         
audioread                     2.1.9          
autograd                      1.3            
Babel                         2.9.0          
backcall                      0.2.0          
beautifulsoup4                4.6.3          
bleach                        3.2.1          
blis                          0.4.

# RENEWABLES ACTION DATASET ANNOTATIONS 


## Electric Vehicles
* Source: Center for Sustainable Energy (2020). Massachusetts Department of  Energy Resources Massachusetts Offers Rebates for Electric Vehicles, Rebate Statistics.  
* Retrieved 09/08/2020 from: https://mor-ev.org/program-statistics
* Data last updated 08/21/2020. Data date range includes 06/19/2014 - 08/15/2020.
* Sectors: Residential.


## Residential Air-source Heat Pumps (ASHP)

* Source: Massachusetts Clean Energy Center (2020). Air Source Heat Pump Program - Residential Projects.
* Retrieved 09/08/2020 from: http://files-cdn.masscec.com/ResidentialASHPProjectDatabase%2011.4.2019.xlsx
* Data last updated 11/04/2019. Data date range includes 12/26/2014 - 10/23/2019.
* Sectors: Residential.


## Ground-source Heat Pumps (GSHP)

* Source: Massachusetts Clean Energy Center (2020). Ground Source Heat Pump Program - Residential & Small-Scale Projects Database.
* Retrieved 09/08/2020 from: http://files-cdn.masscec.com/get-clean-energy/govt-np/clean-heating-cooling/ResidentialandSmallScaleGSHPProjectDatabase.xlsx 
* Data last updated June 2020. Data date range includes 01/02/2015 - 06/09/2020.
* Sectors: Residential, Small Commercial.

## Production Tracking System for Solar Photovoltaic Report (PV in PTS)

* Source: Massachusetts Clean Energy Center
* According to a [September 2017 Department of Public Utilities Report](https://fileservice.eea.comacloud.net/FileService.Api/file/FileRoom/9174030), "On a monthly basis, DOER and MassCEC compile data from the production tracking system to produce the MA PV Report, which is a publicly available document.  The MA PV Report is available electronically at
http://files.masscec.com/uploads/attachments/PVinPTSwebsite.xlsx."  However, the file at that URL does not seem to have been updated since November 2019.  We analyze it here, but need to search for a source of ongoing updated data.
* Sectors: Residential, Commercial, Institutional



# Mount input data files from Google Drive

Instructions from [this tutorial](https://colab.research.google.com/notebooks/io.ipynb)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!ls /content/drive/Shareddrives/MEinternal-DataWG/RAD\ Renewable\ Actions\ Data

'05-04-2020 Data Group Agenda.gdoc'
 Analyses
 Code
'Data Downloads'
'Data Fields Overview.xlsx'
'Data Not Yet Included'
 MassData
 merge-data-AH.ipynb
 merge-data-v3.ipynb
'[old] merge-data.ipynb'
 PVinPTSwebsite.xlsx
'Renewable Actions Dataset Instructions.gdoc'
'Renewables Action Dataset Annotations.gdoc'
'ResidentialASHPProjectDatabase 11.4.2019.xlsx'
 SAMPLE-all-actions-data.xlsx
'Untitled document.gdoc'
'Zip Code Community.xlsx'
'Zip Code Muni Name.drawio'


# Dataset filename mappings

In [3]:
import pandas as pd
import os
import re

In [4]:
data_dir = "/content/drive/Shareddrives/MEinternal-DataWG/RAD Renewable Actions Data"
data_files = {
    "zip_code_community": os.path.join(data_dir, "Zip Code Community.xlsx"),
    "residental_ashp_project_database": os.path.join(data_dir, "ResidentialASHPProjectDatabase 11.4.2019.xlsx"),
    "pv_in_pts_website": os.path.join(data_dir, "PVinPTSwebsite.xlsx")
}
data_files

{'pv_in_pts_website': '/content/drive/Shareddrives/MEinternal-DataWG/RAD Renewable Actions Data/PVinPTSwebsite.xlsx',
 'residental_ashp_project_database': '/content/drive/Shareddrives/MEinternal-DataWG/RAD Renewable Actions Data/ResidentialASHPProjectDatabase 11.4.2019.xlsx',
 'zip_code_community': '/content/drive/Shareddrives/MEinternal-DataWG/RAD Renewable Actions Data/Zip Code Community.xlsx'}

# Work out robust logic for cleaning zipcodes

In [5]:
# Load Zip Code Municipality -- 
# @params: file_name, column_name
zipcodes = pd.read_excel(data_files["zip_code_community"], 'Villages to Muni with Zip')
municipalities = pd.read_excel(data_files["zip_code_community"], '351 Mass Munis')

In [6]:
zipcodes.head()

,Zip Code,City,Municipality,County,Unnamed: 4,Unnamed: 5
0,1001.0,Agawam,Agawam,Hampden,NaN,NaN
1,1002.0,Amherst,Amherst,Hampshire,NaN,NaN
2,1003.0,Amherst,Amherst,Hampshire,NaN,NaN
3,1004.0,Amherst,Amherst,Hampshire,NaN,NaN
4,1005.0,Barre,Barre,Worcester,NaN,NaN


In [7]:
zipcodes.dtypes

Zip Code        float64
City             object
Municipality     object
County           object
Unnamed: 4       object
Unnamed: 5       object
dtype: object

In [8]:
pathalogical_test_case = pd.Series(data = [2186.0, "2186", 2186, "01545-", "   1590 ", "01545-2790", "12345-123", "02128y", 176.2])
correct_output = pd.DataFrame(data={
    "cleaned": ["02186", "02186", "02186", "01545", "01590", "01545-2790", "12345-123", "02128y", 176.2],
    "valid": [True, True, True, True, True, True, False, False, False]
})

In [9]:
pathalogical_test_case.dtypes

dtype('O')

In [10]:
valid_zipcode_regex = r"^([0-9]{3,5})(?:[.]0)?(?:-([0-9]{4})|-)?$"
res = pathalogical_test_case.astype(str).str.strip().str.extract(valid_zipcode_regex)
res[0].str.zfill(5)

0    02186
1    02186
2    02186
3    01545
4    01590
5    01545
6      NaN
7      NaN
8      NaN
Name: 0, dtype: object

In [11]:
cleaned_zipcode_series = \
res[0].str.zfill(5)\
  .str.cat(res[1], sep="-", na_rep="")\
  .str.rstrip("-")
cleaned_zipcode_series

0         02186
1         02186
2         02186
3         01545
4         01590
5    01545-2790
6              
7              
8              
Name: 0, dtype: object

In [12]:
valid_zipcode_series = cleaned_zipcode_series.str.match(valid_zipcode_regex)
valid_zipcode_series

0     True
1     True
2     True
3     True
4     True
5     True
6    False
7    False
8    False
Name: 0, dtype: bool

In [13]:
def clean_zipcodes(zip_series):
  """Standardize zipcodes in a pandas series
  
  Pandas will likely load zipcodes from an excel file as an object series
  mixing string and numeric types.  This function will cast all entries to 
  string types, strip whitespace, left pad to a minimum of 5 characters with zeros, then 
  validate the entry contains a valid zipcode using a regular expression.

  Return: pandas.DataFrame
    - Pandas series of cleaned zipcodes: str
    - Pandas series of valid zipcode indicators: boolean
  """
  # Valid zipcodes are 3-5 numeric digits, followed by an optional dash and 4 more digits, or an optional ".0" if the data has been cast as floats.
  valid_zipcode_regex = r"^([0-9]{3,5})(?:[.]0)?(?:-([0-9]{4})|-)?$"
  #The extract function will match this pattern, and extract the zip5 group into column 0 and the zip4 group into column 1.  NaN if group is not present or pattern isn't matched.
  res = zip_series.astype(str).str.strip().str.extract(valid_zipcode_regex)

  cleaned_zipcode_series = \
                            res[0].str.zfill(5)\
                            .str.cat(res[1], sep="-", na_rep="")\
                            .str.rstrip("-")

  valid_zipcode_series = cleaned_zipcode_series.str.match(valid_zipcode_regex)
  
  #Replace invalid value rows with original inputs
  cleaned_zipcode_series.loc[~valid_zipcode_series] = zip_series[~valid_zipcode_series]
  return pd.DataFrame(data={"cleaned": cleaned_zipcode_series, "valid": valid_zipcode_series})



In [14]:
z_df = clean_zipcodes(pathalogical_test_case)

In [15]:
if pd.testing.assert_frame_equal(z_df, correct_output) is None:
  print("Success!")

Success!


# Load Air Source Heat Pump (ASHP) program data and Solar program data

Load from excel into pandas dataframes and peek at it for sanity.

In [18]:
#PTS solar data next is the important one, tracking data

# One thing for these datasets is that it's possible to read all the ZipCode fields as strings, and in that case it would not get rid of the '0' prefix in front of all Zip Codes
f_ashp = pd.read_excel(data_files["residental_ashp_project_database"], 'Sheet1', skiprows=3)
df_ashp = f_ashp.drop([0]) #remove first null row for formatting purposes

#Read PVinPTS Data
df_pv = pd.read_excel(data_files["pv_in_pts_website"], 'PV in PTS', skiprows=7)

In [ ]:
df_ashp.head()

,Date Rebate Payment Approved by MassCEC,Site City/Town,Site Zip Code,Installer Company Name,Heating Fuel Being Replaced,Cooling Type Being Replaced,# of Outdoor Units,# of Indoor Units,Capacity of Heat Pumps at 5°F,Single- Head Heat Pump #1,Single- Head Heat Pump #2,Single- Head Heat Pump #3,Multi-Head Heat Pump #1,Multi-Head Heat Pump #2,Multi-Head Heat Pump #3,Total System Costs,Receiving an Income-Based Adder?,Rebate Amount
1,2019-10-23,CENTERVILLE,2632,"Seaside Gas Service, Inc",Natural Gas,None,1,3,25000.761761,NaN,NaN,NaN,NaN,NaN,NaN,11575,0,1302.08
2,2019-10-23,NORTHFIELD,1360,Arctic Refrigeration,Pellet Stove,2 window units,1,1,20300.000000,NaN,NaN,NaN,NaN,NaN,NaN,4050,0,625
3,2019-10-16,West Tisbury,2575,Nelson Mechanical Design Inc,Propane,Window Unit(s),2,-,57200.400000,,,,Mitsubishi MXZ-3C30NAHZ2,Mitsubishi MXZ-3C30NAHZ2,NaN,25560,No - Not Applicable,2000
4,2019-10-16,FITCHBURG,1420,Royal Steam Heater Co.,Natural Gas,Window fan,4,15,92997.920266,NaN,NaN,NaN,NaN,NaN,NaN,44352,0,6200
5,2019-10-09,Haverhill,01835,Climate Zone,Oil,Window Unit(s),1,NaN,28600.000000,0,0,0,Mitsubishi-MXZ-3C30NAHZ2,0,0,13700,No - Not Applicable,1191.67


Looks like data describing individual installation projects. 

In [19]:
df_pv.head()

,"Capacity \n(DC, kW)",Date In Service,Total Cost with Design Fees,Total Grant,City,Zip,County,Program Name,Facility Type,Installer,Module Manufacturer,Inverter Manufacturer,Meter Manufacturer,Utility,3rd Party Owner,SREC Eligible,Estimated Annual Production (kWhr)
0,1077.48,2019-08-08,3462750.0,0.0,Boston,2128,Suffolk,Non-RET Funded Grants,Industrial,ECA Solar,Jinko Solar,Solectria;SolarEdge Technologies,Elkor Technologies,NSTAR (DBA EverSource),N,Y,1345600.0
1,218.28,2019-07-30,600000.0,0.0,Pittsfield,1201,Berkshire,Non-RET Funded Grants,Commercial / Office,"BVD, LLC",Seraphim Solar System,Solectria;Solectria;Solectria,Elkor Technologies,WMECO (DBA EverSource),N,Y,288557.0
2,296.00,2019-07-09,760806.0,0.0,Great Barrington,1230,Berkshire,Non-RET Funded Grants,Commercial / Office,Solect Energy Development LLC,LG Electronics,HiQ Solar,eGauge,National Grid,Y,Y,309800.0
3,1408.96,2019-06-27,4064405.0,0.0,Millbury,1527,Worcester,Non-RET Funded Grants,Community Solar,M&amp;W Energy,Hansol Technics,Sungrow Power,Accuenergy,National Grid,N,Y,1760000.0
4,657.00,2019-06-21,738468.0,0.0,Walpole,2081,Norfolk,Non-RET Funded Grants,Industrial,ECA Solar,LG Electronics,Solectria;Solectria;Solectria,Elkor Technologies,NSTAR (DBA EverSource),N,Y,804168.0


# Clean zipcode data

In [24]:
x = clean_zipcodes(df_ashp['Site Zip Code'])
x[~x.valid]

,cleaned,valid
1093,20,False
1590,019081047,False
1606,0212y,False


In [23]:
y = clean_zipcodes(df_pv['Zip'])
y[~y.valid]

,cleaned,valid


## Standardize municipality names

Verify that town names are consistent with the Mass Muni list spelling and are consistent with zipcode where applicable

In [ ]:
#Checks that the data's town name is present in the 351 mass munis list or villages to muni with zip list. 
def matchMuniName(siteCityName, siteZipCode):
    x = municipalities.loc[municipalities['Municipality'] == siteCityName]
    if not x.empty:
        if x.size > 1:
            print("There are multiple municipalities for,", siteCityName)
        return x.iloc[0]['City']
    else:
        y = zipcodes.loc[zipcodes['City'] == siteCityName]
        if y.size > 1: 
            print("There are multiple municipalities for", siteCityName)
        return y.iloc[0]['City']
        #also use zip codes
    return "NF"

matchMuniName("Florence", '02138')

There are multiple municipalities for Florence


'Florence'

In [ ]:
#Checks that the data's town name is present in the 351 mass munis list or villages to muni with zip list. 
def matchMuniName(siteCityName, siteZipCode):
    x = municipalities.loc[municipalities['Municipality'] == siteCityName]
    if not x.empty:
        # print(x.values)
        return x.values.astype('str')
    else:
        y = zipcodes.loc[zipcodes['City'] == siteCityName]
        print(y)
        return y
        #also use zip codes
    return "NF"

matchMuniName("Boston", '02138')

array([['Boston']], dtype='<U6')

In [ ]:
#Implement and use the predefined matchMuniName function. 
def processZipCode(df, zipCodeColumnName, townColumnName, newMuniName):
    counter = 0
    for index,row in df.iterrows():
        if row[townColumnName]:
            if not df.loc[df[townColumnName].str.lower() == row['Site City/Town'].lower()].empty:
                print(row['Site City/Town'])
        #first use the 351 Mass Munis (the municipalities dataframe)
        #then use the Villages to Munu with Zip (the zipcodes data frame) to match if value is not present in mass munis. 
            
            #else: if the Site City/Town is not in the municipality list, then choose a different 

        #if there is a zip code 
            #if not a zip code 
                #check multi zip code
                    #find address
                #use matchMuniName()

processZipCode(df_ashp, "", "", "")

In [ ]:
#Format Technology

df_ashp = df_ashp.assign(Technology = 'Air Source Heat Pumps')
#Do the Same for other data sets

print(df_ashp)

Date Rebate Payment Approved by MassCEC  Site City/Town Site Zip Code  \
1                                  2019-10-23     CENTERVILLE          2632   
2                                  2019-10-23      NORTHFIELD          1360   
3                                  2019-10-16    West Tisbury          2575   
4                                  2019-10-16       FITCHBURG          1420   
5                                  2019-10-09       Haverhill         01835   
...                                       ...             ...           ...   
19960                              2015-01-14         Bedford          1730   
19961                              2015-01-08     Westborough          1581   
19962                              2015-01-08  Yarmouth Port           2675   
19963                              2015-01-06     LEOMINSTER           1453   
19964                              2014-12-26         Amherst          1002   

                 Installer Company Name Heating Fuel Bein

In [ ]:
#Date and Time Formatting

def processDates(df, dateColumn, yearName, monthName):
    for index, row in df.iterrows():
        date = row[dateColumn]
        row[yearName] = date.year
        row[monthName] = date.month
    #select the year, month from these columns

processDates(df_ashp,'Date Rebate Payment Approved by MassCEC', 'Year', 'Month')

In [ ]:
#Quantity: Use Pandas df['Application Number'].unique() to find all unique values

print(df_ashp['Site City/Town'].unique())

['CENTERVILLE' 'NORTHFIELD' 'West Tisbury' ... 'Shelburne Center'
 'Erving ' 'N Carver ']


In [ ]:
#Rebates

#Assign rebates directy from the reabtes in the ASHP data -- read all as integer or dollar values

In [ ]:
#"Snap to Grid"
# Match Column names and copy over data


In [ ]:
#Data Fields for Final Dataset 
dataFields = ['Municipality', 'Zip Code', 'Tech', 'Year', 'Month', 'Quantity', 'Average Cost',
    'Total Cost', 'Total Rebates', 'Average Rebate', 'Count Income-Eligible']

df_final = pd.DataFrame(columns=dataFields)

print(df_final)

#to_csv